In [34]:
import tensorflow as tf
tf.random.set_seed(42)
from kerasbeats import prep_time_series, NBeatsModel
from tensorflow import keras
import numpy as np
import pandas as pd


def create_nbeat_mlp(num_columns, num_labels, lookback, horizon, hidden_units, dropout_rates, batch_size, ls=1e-2, lr=1e-3, ):
    nbeats = NBeatsModel(model_type = 'generic', lookback = lookback, horizon = horizon,
                         learning_rate = lr, batch_size = batch_size,
                         num_generic_neurons = hidden_units[0]) # set as default
    nbeats.build_layer()
    time_input = keras.layers.Input(shape = (lookback * horizon, ))
    x_nb = nbeats.model_layer(time_input)

    xcons = keras.layers.Input(shape = (num_columns, ))
    x = keras.layers.Concatenate()([xcons, x_nb])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)

    for i in range(1, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i])(x)

    out = tf.keras.layers.Dense(num_labels, name = 'action')(x)
    model = tf.keras.models.Model(inputs = [time_input, xcons], outputs = out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss = {'action' : tf.keras.losses.MeanSquaredError()},
                  metrics = {'action' : tf.metrics.MeanSquaredError(name = 'mse')})
    return model

In [15]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from itertools import islice
import tscv

def generate_label(df, threshold = 0.002):
    df['label'] = 0
    df.loc[(df['target_15m'] <= -1*threshold), 'label'] = 1
    df.loc[(df['target_15m'] >= threshold), 'label'] = 2
    return df

def get_na_features(df, train_features):
    tmp = pd.DataFrame(df[train_features].isnull().sum())
    tmp = tmp[tmp[0] > 0].reset_index()
    tmp.columns = ['feat', 'cnt']
    tmp = tmp.sort_values('cnt')
    feat_groups = dict(tmp.groupby('cnt')['feat'].agg(lambda x: list(x)))
    return feat_groups

In [18]:
def normalize_columns_infer(df, means, stds):
    features = means.index.tolist()
    for feature in features:
        mean = means.loc[feature]
        std = stds.loc[feature]
        df[feature] = (df[feature] - mean) / std
    return df

In [17]:
from pathlib import Path
directory = 'spot_data_11_25'
date = '11_25'
lib = Path(f"../output/{directory}").mkdir(parents=True, exist_ok=True)
window = 30
file = f'target_{window}m_feature.csv'
corr = pd.read_csv(f'../output/feature_corr/{file}', header = 0, index_col = 0)

train_features = corr.iloc[:50].index.tolist()

In [36]:
train = False
df = pd.read_feather('../data/df_btc_with_features_5m_spot.feather')

df = df.loc[df['token'] == 'BTCUSDT'].reset_index(drop = True)

df = df.sort_values(by='open_time', ignore_index=True)

df['close_5m'] = df['close'].shift(-1)
df['close_t'] = df['close'].copy()
df['target_5m'] = df['close'].pct_change(1)

start_time = df['open_time'].min()
end_time = df['open_time'].max()
dates = df['open_time'].unique()
n = len(dates)
train_idx = int(0.7 * n)
valid_idx = int(0.9 * n)
train_end = dates[train_idx]
valid_end = dates[valid_idx]

train_df = df.loc[df['open_time'] < train_end].reset_index(drop=True)
valid_df = df.loc[(train_end <= df['open_time']) & (df['open_time'] < valid_end)].reset_index(drop=True)

train_df = pd.concat([train_df, valid_df], axis = 0)

test_df = df.loc[(df['open_time'] >= valid_end)].reset_index(drop=True)

valid_df = test_df.copy()

groups = pd.factorize(
    train_df['open_time'].dt.day.astype(str) + '_' + train_df['open_time'].dt.month.astype(str) + '_' + train_df[
        'open_time'].dt.year.astype(str))[0]

cv = tscv.PurgedGroupTimeSeriesSplit(
    n_splits=5,
    group_gap=31,
)

from joblib import load, dump
date = f'11_25'
directory = f'spot_data_11_25'
lookback = 5
horizon = 100
means = load(f'../output/{directory}/means_nbeats_huber_l={lookback}_h={horizon}.joblib')
std = load(f'../output/{directory}/std_return_nbeats_l={lookback}_h={horizon}.joblib')
train_features_test = load(
    f'../output/{directory}/train_features_test_return_nbeats_l={lookback}_h={horizon}.joblib')

valid_df = normalize_columns_infer(valid_df, means, std)

lookback = 5
horizon = 100
batch_size = 4096
params = {'num_columns': len(train_features_test),
          'num_labels': 100,
          'lookback' : lookback,
          'horizon' : horizon,
          'batch_size' : batch_size,
          'hidden_units': [200, 200, 300, 256],
          'dropout_rates': [0.42409238408801436, 0.10431484318345882,
                            0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448],
          'ls': 0,
          'lr': 1e-3,
          }

x_val, y_val = prep_time_series(valid_df['target_5m'], lookback = lookback, horizon = horizon)
cutoff_val = valid_df.shape[0] - x_val.shape[0]

x_val_cons = valid_df[train_features_test].iloc[cutoff_val:]
print(f'Shape of X_const is {x_val_cons.shape}, x_tr is {x_val.shape}, y_tr is {y_val.shape}')
fold = 4
model = create_nbeat_mlp(**params)
model.load_weights(f'../output/{directory}/NBEATS_MSE_{fold}_returns{horizon}m_{lookback}m_{date}.hdf5')

preds = model.predict([x_val, x_val_cons])


Shape of X_const is (65150, 50), x_tr is (65150, 500), y_tr is (65150, 100)
2036/2036 [==============================] - 31s 15ms/step


In [46]:
x_val_cons = valid_df[['close_t', 'close_5m']].iloc[cutoff_val:].reset_index(drop = True)

In [50]:
from tqdm import tqdm
ypred = []
ytrue = []
for i, row in tqdm(x_val_cons.iterrows()):
    rets = preds[i, :]
    prices = (1 + rets) * row['close_t']
    ypred.append(prices)
    target = x_val_cons['close_5m'].iloc[i:i+100]
    ytrue.append(target)

65150it [02:34, 422.65it/s] 


In [51]:
def calculate_weighted_rmse(predictions: np, actual: np) -> float:
    predictions = np.array(predictions)
    actual = np.array(actual)

    k = 0.001

    weights = np.exp(-k * np.arange(len(predictions)))
    weighted_squared_errors = weights * (predictions - actual) ** 2
    weighted_rmse = np.sqrt(np.sum(weighted_squared_errors) / np.sum(weights))

    return weighted_rmse

In [55]:
rmse = []
for i in tqdm(range(1, len(ypred)-100)):
    ypred_i = ypred[i]
    ytrue_i = ytrue[i]
    score = calculate_weighted_rmse(ytrue_i, ypred_i)
    rmse.append(score)

100%|██████████| 65049/65049 [00:01<00:00, 36644.75it/s]


In [56]:
rmse_df = pd.DataFrame(rmse)
# pred_df = pd.DataFrame(ypred)
# true_df = pd.DataFrame(ytrue)

KeyboardInterrupt: 

In [72]:
test_x = x_val[-1, :].reshape(1, -1)
test_x_cons = valid_df[train_features_test].iloc[-1, :].values.reshape(1, -1)

In [78]:
preds = model.predict([test_x, test_x_cons])


1/1 [==============================] - 0s 29ms/step


In [79]:
preds.reshape(-1, 1)

array([[-2.7423457e-04],
       [-2.3064767e-03],
       [-1.4398299e-03],
       [-1.3466255e-04],
       [-6.1064749e-04],
       [-1.0027986e-03],
       [ 1.5606404e-03],
       [ 2.3287081e-04],
       [ 7.3816400e-04],
       [-3.8709759e-04],
       [-3.4602184e-04],
       [-8.0463884e-04],
       [-1.0930328e-03],
       [-5.1470066e-04],
       [-1.9291659e-03],
       [-5.9225300e-04],
       [-9.9845883e-04],
       [-7.2583638e-04],
       [ 2.5094178e-04],
       [-5.6439487e-04],
       [-2.5192969e-03],
       [ 1.7120889e-04],
       [-3.9061165e-04],
       [ 1.5866286e-03],
       [ 5.5073638e-04],
       [-1.1011825e-03],
       [ 1.2349584e-03],
       [ 3.5656849e-06],
       [ 8.2784583e-04],
       [-5.6279369e-04],
       [-3.0728395e-04],
       [ 9.1687473e-04],
       [-4.0490413e-05],
       [ 3.9559440e-05],
       [ 2.9032512e-04],
       [-2.7043046e-05],
       [-4.9188617e-05],
       [ 9.6712151e-04],
       [ 6.1888609e-04],
       [-1.4302582e-03],


In [76]:
rmse_df.mean(axis = 0)

0    166.624309
dtype: float64